In [2]:
from wrangle import wrangle
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
%config InlineBackend.figure_format='retina'

In [3]:
X_train = pd.read_csv('data/train_features.csv')
X_test = pd.read_csv('data/test_features.csv')
y_train = pd.read_csv('data/train_labels.csv')['charged_off']
sample_submission = pd.read_csv('data/sample_submission.csv')

X_train.shape, X_test.shape, y_train.shape

((1309457, 103), (26724, 103), (1309457,))

In [4]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=40000, test_size=40000,
    random_state=42, stratify=y_train)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((40000, 103), (40000, 103), (40000,), (40000,))

In [5]:
X_train = wrangle(X_train)
X_val = wrangle(X_val)
X_train.shape, X_val.shape

((40000, 98), (40000, 98))

In [6]:
# Catboost

cat_features = list(X_train.select_dtypes('object').columns)

ct = CatBoostClassifier(iterations=10,depth=6,learning_rate=1,
                        nan_mode='Max',thread_count=-1,
                        cat_features=cat_features,verbose=False,
                        l2_leaf_reg=1)

ct.fit(X_train,y_train)

y_pred_proba = ct.predict_proba(X_val)[:,1]
print('Validation ROC AUC:', roc_auc_score(y_val, y_pred_proba))

Validation ROC AUC: 0.7040768332533571


In [16]:
# XGBoost

X_train_encoded = ce.OrdinalEncoder().fit_transform(X_train)
X_val_encoded = ce.OrdinalEncoder().fit_transform(X_val)

xgb = XGBClassifier(learning_rate=1,max_depth=6,n_estimators=15, 
                    nthread=-1)

xgb.fit(X_train_encoded, y_train)
y_pred_proba = xgb.predict_proba(X_val_encoded)[:,1]
print('Validation ROC AUC:', roc_auc_score(y_val, y_pred_proba))

Validation ROC AUC: 0.6055037805328715
